# XOR ANN implemented in Pure Python
This ANN is based on the discussion in section 4 of _"From Matrices to Models."_ In this notebook, you'll find the code needed to manually implement the ANN that solves XOR, along with annotations and comments that reference parts of the paper.



Firstly, we setup our imports, and we define the two activation functions we'll use later, which are ReLU (Equation 3.4) and sigmoid (Equation 3.5). Notice that we define both to operate on _each entry in a vector or matrix_—this is key.

In [1]:
import numpy as np
from numpy import floating

# Equation (3.4)
def relu(z) -> np.ndarray:
    return np.maximum(0, z)

# The derivative of Equation (3.4)
def relu_derivative(z: np.ndarray) -> np.ndarray:
    return (z > 0).astype(float)

# Equation (3.5)
def sigmoid(z: np.ndarray) -> np.ndarray:
    return 1 / (1 + np.exp(-z))

# The derivative of Equation (3.5)
def sigmoid_derivative(z: np.ndarray) -> np.ndarray:
    return sigmoid(z) * (1 - sigmoid(z))

Next, we define our:
- input data (`X`), a matrix;
- expected outputs (`y`), a vector;
- and our hyperparameters.

The hyperparameters describe the structure of the network (how many neurons in each layer) and learning algorithm parameters (the learning rate and number of epochs to train for). Specifically, `learning_rate` is $\eta$ in Equation 4.3, and `training_epochs` represents how many times we should run the training algorithm. These numbers both must be tuned on a network-by-network basis—trial and error is almost guaranteed. Specifically, training too much can lead to an overfit algorithm (and too little leads to an underfit algorithm) [9].

In [2]:
# Setup input/output data for xor
X: np.ndarray = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y: np.ndarray = np.array([[0], [1], [1], [0]])

# Hyperparameters
input_size: int = 2             # 2 neurons in the input layer
hidden_layer_size: int = 2      # 2 neurons in the hidden layer
output_size: int = 1            # 1 neuron in the output layer

learning_rate: float = 0.1     # eta in equation 4.3

training_epochs: int = 10000    # how many times to run the training algorithm (fine tune 
                                # this number, since the network can over- or under-fit if
                                # this number is too large or small—see [9] for more info)

Now, we initialize the weights and biases for the network to random values. `np.random.randn(rows, columns)` generates a random `ndarray` (matrix or vector) of numbers. Then, as described in Equation (3.3), transitions between layers can be thought of as linear transformations, such that:
-  $T_1 : \mathbb{R}^2 \rightarrow \mathbb{R}^2$ (input layer $\rightarrow$ hidden layer) has weight matrix $W_1$ and bias vector $b_1$
- $T_2 : \mathbb{R}^2 \rightarrow \mathbb{R}$ (hidden layer $\rightarrow$ output layer) has weight matrix $W_2$ and bias vector $b_2$

We scale the random numbers—in matrices $W_1$ and $W_2$, we use what is called Xavier initialization. This scales the weights so that the variance of the activations is the same across layers, helping to ensure that the network converges to a global minimum during training [10].

In [3]:
# Initialize weights and biases

np.random.seed(38)  # for reproducibility

# T_1 : input layer -> hidden layer (R^2 -> R^2)
W1: np.ndarray = np.random.randn(input_size, hidden_layer_size) * np.sqrt(2. / input_size) # Scale the weights
                                                                    # so that the variance of activations is
                                                                    # the same across layers (Xavier Initialization)
b1: np.ndarray = np.random.randn(1, hidden_layer_size) * 0.1

# T_2 : hidden layer -> output layer (R^2 -> R)
W2: np.ndarray = np.random.randn(hidden_layer_size, output_size) * np.sqrt(2. / input_size)
b2: np.ndarray = np.random.randn(1, output_size) * 0.1

Next, it's time to train the network. This is done in a two-step process:
1. We do a forward pass, computing the current network estimation. We calculate the cost function—in the case of this network, MSE: $J(\textbf{w}, b) = \frac{1}{4}\sum_{\textbf{d} \in D} (f^*(\textbf{d}) - f(\textbf{d}, \textbf{w}, b))$ (Equation 4.1)
        
2. We perform batch gradient descent and backpropagation, described by Equation 4.3:
$\textbf{w} = \textbf{w} - \eta \cdot \nabla J(\textbf{w})$

In [4]:
# Train the network
for epoch in range(training_epochs):
    # Forward pass
    z1: np.ndarray = np.dot(X, W1) + b1 # Evaluate the inside of the ReLU function
    a1: np.ndarray = relu(z1) # Apply ReLU
    
    z2: np.ndarray = np.dot(a1, W2) + b2
    a2: np.ndarray = sigmoid(z2)
    
    error: np.ndarray = y - a2
    cost: floating = np.mean(error**2)
    
    # Backpropagation (Equation 4.3)
    d_a2: np.ndarray = error * sigmoid_derivative(a2)            # Estimate gradient at output
    d_W2: np.ndarray = np.dot(a1.T, d_a2)                        # Get the weight update for W2
    d_b2: np.ndarray = np.sum(d_a2, axis=0, keepdims=True)       # Get the bias update for b2
    
    d_a1: np.ndarray = np.dot(d_a2, W2.T) * relu_derivative(a1)  # Estimate gradient at hidden layer
    d_W1: np.ndarray = np.dot(X.T, d_a1)                         # Get the weight update for W1
    d_b1: np.ndarray = np.sum(d_a1, axis=0, keepdims=True)       # Get the bias update for b1
    
    # Update the weights and biases
    W1 += learning_rate * d_W1
    b1 += learning_rate * d_b1
    W2 += learning_rate * d_W2
    b2 += learning_rate * d_b2
    
    # Print the cost occasionally
    if epoch % 1000 == 0:
        print(f'Epoch: {epoch}, Cost: {cost}')

Epoch: 0, Cost: 0.2837071413735631
Epoch: 1000, Cost: 0.11898073301403354
Epoch: 2000, Cost: 0.0009480482553462136
Epoch: 3000, Cost: 0.00020454082651267882
Epoch: 4000, Cost: 8.378823534111242e-05
Epoch: 5000, Cost: 4.4712507442895376e-05
Epoch: 6000, Cost: 2.748404894141129e-05
Epoch: 7000, Cost: 1.8532466873386784e-05
Epoch: 8000, Cost: 1.329569505916439e-05
Epoch: 9000, Cost: 9.983495112767322e-06


Our network is trained and good to go! Now, we test it with all the values we trained it on (Equations 4.12 and 4.13).

In [5]:
# Testing
print("Predictions after training:")
for i in range(len(X)):
    z1: np.ndarray = np.dot(X[i], W1) + b1
    a1: np.ndarray = relu(z1)
    z2: np.ndarray = np.dot(a1, W2) + b2
    a2: np.ndarray = sigmoid(z2)
    print(f"Input: {X[i]} -> Predicted Output: {a2.round()} (Raw: {a2})")

Predictions after training:
Input: [0 0] -> Predicted Output: [[0.]] (Raw: [[0.0006837]])
Input: [0 1] -> Predicted Output: [[1.]] (Raw: [[0.99612045]])
Input: [1 0] -> Predicted Output: [[1.]] (Raw: [[0.99612045]])
Input: [1 1] -> Predicted Output: [[0.]] (Raw: [[0.00068277]])
